In [1]:
import torch
import wandb
import os
import math
from color.CEConv.models.resnet_variational import ResNet18 as ResNet18_partial
from color.generate_data import generate_102flower_data
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import yaml
from easydict import EasyDict
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
from tqdm import tqdm
import seaborn as sns
sns.set()
plt.rc('axes', labelsize=15)   # x,y축 label 폰트 크기
plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)

/mnt/home/yegonkim/.conda/envs/brad/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# run_path = "kim-hyunsu/partial_equiv/0vh427nx"
file = "/mnt/home/yegonkim/home/partial_equiv_project/partial_equiv/wandb/run-20240324_193430-1ca72q92/files/config.yaml"
with open(file) as f:
    used_args = yaml.safe_load(f)
args = dict()
for k in used_args:
    v = used_args[k]
    if isinstance(v, dict) and not k.startswith("_"):
        if "." in k:
            temp = args
            for _k in k.split(".")[:-1]:
                if temp.get(_k) is None:
                    temp[_k] = dict()
                temp = temp[_k]
            temp[k.split(".")[-1]] = v["value"]
        else:
            args[k] = v["value"]
args = EasyDict(args)
print(args)
torch.manual_seed(args.seed)
np.random.seed(args.seed)

os.environ["WANDB_MODE"] = "dryrun"
os.environ["HYDRA_FULL_ERROR"] = "1"

wandb.init(
    project="partial_equiv",
    entity="kim-hyunsu",
    reinit=True,
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


{'type': 'color', 'dataset': 'Flowers102', 'pretrained': None, 'device': 'cuda', 'seed': 2024, 'comment': '', 'no_workers': 1, 'flops': False, 'train': {'do': True, 'epochs': 700, 'batch_size': 64, 'valid_every': 10, 'weight_decay': 0.01, 'lr': 0.0002, 'lr_probs': 2e-05, 'lamda': 0.1, 'lamda2': 0.01}, 'model': {'architecture': 'resnet18', 'rot': 3, 'partial': False, 'variational': True, 'version': 'v1.2', 'maxpool': False, 'insta': False, 'insta_params': {'num_samples': 1, 'lambda_entropy': 0.0001, 'h_min': -1.5, 'h_max': 2}}, 'wandb': {'entity': 'kim-hyunsu', 'project': 'partial_equiv', 'mode': 'online'}}


In [3]:
flower_names = ['pink primrose'
,'hard-leaved pocket orchid'
,'canterbury bells'
,'sweet pea'
,'english marigold'
,'tiger lily'
,'moon orchid'
,'bird of paradise'
,'monkshood'
,'globe thistle'
,'snapdragon'
,"colt's foot"
,'king protea'
,'spear thistle'
,'yellow iris'
,'globe-flower'
,'purple coneflower'
,'peruvian lily'
,'balloon flower'
,'giant white arum lily'
,'fire lily'
,'pincushion flower'
,'fritillary'
,'red ginger'
,'grape hyacinth'
,'corn poppy'
,'prince of wales feathers'
,'stemless gentian'
,'artichoke'
,'sweet william'
,'carnation'
,'garden phlox'
,'love in the mist'
,'mexican aster'
,'alpine sea holly'
,'ruby-lipped cattleya'
,'cape flower'
,'great masterwort'
,'siam tulip'
,'lenten rose'
,'barbeton daisy'
,'daffodil'
,'sword lily'
,'poinsettia'
,'bolero deep blue'
,'wallflower'
,'marigold'
,'buttercup'
,'oxeye daisy'
,'common dandelion'
,'petunia'
,'wild pansy'
,'primula'
,'sunflower'
,'pelargonium'
,'bishop of llandaff'
,'gaura'
,'geranium'
,'orange dahlia'
,'pink-yellow dahlia?'
,'cautleya spicata'
,'japanese anemone'
,'black-eyed susan'
,'silverbush'
,'californian poppy'
,'osteospermum'
,'spring crocus'
,'bearded iris'
,'windflower'
,'tree poppy'
,'gazania'
,'azalea'
,'water lily'
,'rose'
,'thorn apple'
,'morning glory'
,'passion flower'
,'lotus'
,'toad lily'
,'anthurium'
,'frangipani'
,'clematis'
,'hibiscus'
,'columbine'
,'desert-rose'
,'tree mallow'
,'magnolia'
,'cyclamen '
,'watercress'
,'canna lily'
,'hippeastrum '
,'bee balm'
,'ball moss'
,'foxglove'
,'bougainvillea'
,'camellia'
,'mallow'
,'mexican petunia'
,'bromelia'
,'blanket flower'
,'trumpet creeper'
,'blackberry lily']

In [4]:
### Get samples from test dataset
dataset = generate_102flower_data(size=224)

test_loader = torch.utils.data.DataLoader(
        dataset["test"],
        batch_size=1,
        shuffle=False,
        num_workers=1,
    )

In [5]:
gumbel_no_iterations = math.ceil(len(dataset["train"]) / float(args.train.batch_size))  # Iter per epoch
gumbel_no_iterations = args.train.epochs * gumbel_no_iterations
model = ResNet18_partial(pretrained=False, progress=False, rotations=args.model.rot, num_classes=102,
                groupcosetmaxpool=True, separable=True,
                gumbel_no_iterations=gumbel_no_iterations,
                version=args.model.version
        ).to(args.device)
# load checkpoint
# model_checkpoint = wandb.restore('checkpoint.pt', run_path)
# model.load_state_dict(
#             torch.load(model_checkpoint.name, map_location=args.device)["model"],
#             strict=True,
#         )
model = torch.nn.DataParallel(model)
checkpoint_path = "/mnt/home/yegonkim/home/partial_equiv_project/partial_equiv/wandb/run-20240324_193430-1ca72q92/files/checkpoint.pt"
model.load_state_dict(
            torch.load(checkpoint_path, map_location=args.device)["model"],
            strict=True,
        )
model.eval()


DataParallel(
  (module): ResNet(
    (maxpool): GroupMaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (conv1): CEConv2d(
      3, 55, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1), bias=False
      (filter_conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2))
      (filter_nonlinear): ReLU()
      (filter_conv2): Conv2d(6, 6, kernel_size=(3, 3), stride=(2, 2))
      (filter_linear1): Linear(in_features=6, out_features=1, bias=True)
    )
    (bn1): BatchNorm3d(55, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layers): ModuleList(
      (0): Sequential(
        (0): BasicBlock(
          (bn1): BatchNorm3d(55, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (bn2): BatchNorm3d(55, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (shortcut): Sequential()
          (conv1): CEConv2d(55, 55, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fals

In [6]:
def equiv_error(a, b):
    diff = torch.norm(a-b, dim=-1)
    anorm = torch.norm(a, dim=-1)
    bnorm = torch.norm(b, dim=-1)
    return diff

In [15]:
def get_equiv_error(target_labels):
    error_dict = dict()
    for l in target_labels:
        error_dict[str(l)] = []
    jitterings = torch.arange(0,3)/3-0.5 
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(args.device)
            labels = labels.to(args.device)
            if labels[0] not in target_labels:
                continue
            outputs = model(images)[0]
            pred = torch.argmax(outputs, dim=-1)
            if torch.any(pred != labels):
                continue
            # outputs = torch.softmax(outputs, dim=-1)[labels[0]]
            outputs = torch.softmax(outputs, dim=-1)
            _error_list = []
            for j in jitterings:
                if j == 0:
                    continue
                outputs_T = model(TF.adjust_hue(images, j))[0]
                # outputs_T = torch.softmax(outputs_T, dim=-1)[labels[0]]
                outputs_T = torch.softmax(outputs_T, dim=-1)
                error = equiv_error(outputs, outputs_T)
                _error_list.append(error)
            error = torch.stack(_error_list).mean()
            assert len(error.shape) == 0
            error_dict[str(labels[0])].append(error)
    error_list = []
    for l in target_labels:
        stack = torch.stack(error_dict[str(l)])
        # sorted, _ = torch.sort(stack, dim=0)
        ratio = (stack < 0.6).sum() / len(stack)
        error_list.append(ratio)
    return error_list

In [16]:
target_labels = torch.arange(102).to(args.device)
error_list = get_equiv_error(target_labels)
for l, e in zip(target_labels, error_list):
    print(f"{flower_names[l]}: {e}")
# plot_bar(target_labels, samples)

  0%|          | 0/6149 [00:00<?, ?it/s]

100%|██████████| 6149/6149 [05:02<00:00, 20.32it/s]

pink primrose: 1.0
hard-leaved pocket orchid: 1.0
canterbury bells: 1.0
sweet pea: 1.0
english marigold: 1.0
tiger lily: 1.0
moon orchid: 1.0
bird of paradise: 1.0
monkshood: 1.0
globe thistle: 1.0
snapdragon: 1.0
colt's foot: 1.0
king protea: 1.0
spear thistle: 1.0
yellow iris: 1.0
globe-flower: 1.0
purple coneflower: 1.0
peruvian lily: 1.0
balloon flower: 1.0
giant white arum lily: 1.0
fire lily: 1.0
pincushion flower: 1.0
fritillary: 0.9999999403953552
red ginger: 1.0
grape hyacinth: 1.0
corn poppy: 1.0
prince of wales feathers: 1.0
stemless gentian: 1.0
artichoke: 0.9999999403953552
sweet william: 1.0
carnation: 1.0
garden phlox: 1.0
love in the mist: 1.0
mexican aster: 1.0
alpine sea holly: 1.0
ruby-lipped cattleya: 1.0
cape flower: 1.0
great masterwort: 1.0
siam tulip: 1.0
lenten rose: 1.0
barbeton daisy: 1.0
daffodil: 1.0
sword lily: 1.0
poinsettia: 1.0
bolero deep blue: 1.0
wallflower: 1.0
marigold: 1.0
buttercup: 1.0
oxeye daisy: 1.0
common dandelion: 1.0
petunia: 1.0
wild pan